In [ ]:
def get_stats(ids):
    counts = {}
    for a,b in zip(ids,ids[1:]):
        counts[(a,b)] = counts.get((a,b),0) + 1
    return counts

In [ ]:
get_stats([1,2,1,2,1,3])

{(1, 2): 2, (2, 1): 2, (1, 3): 1}

In [ ]:
def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i<len(ids):
        if i<len(ids)-1 and ids[i]==pair[0] and ids[i+1]==pair[1]:
            new_ids.append(idx)
            i += 2
        else: 
            new_ids.append(ids[i])
            i += 1
    return new_ids

In [ ]:
merge([1,2,1,2,1,3], (1,2), 4)

[4, 4, 1, 3]

In [ ]:
class BasicTokenizer:
    """implememt train, encode, decode a string"""
    def __int__(self):
        self.merges = {} # int, int -> int
        self.vocab = {} # int -> bytes

    def train(self,s:str, vocab_size:int):
        ids = list(s.encode('utf-8')) # list of ids in range 0, ...,255
        assert vocab_size>255
        n_merges = vocab_size - 256
        merges = {}
        vocab = {i:bytes([i]) for i in range(256)}
        idx = 256
        for _ in range(n_merges):
            stats = get_stats(ids)
            pair = max(stats, key=stats.get)
            ids = merge(ids,pair,idx)
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]]+vocab[pair[1]]
            idx+=1
        self.merges = merges
        self.vocab = vocab

    def decode(self,ids)-> str:
        b = b"".join(self.vocab[i] for i in ids)
        return b.decode("utf-8")

    def encode(self,s:str)->list[int]:
        ids = list(s.encode('utf-8'))
        while len(ids)>=2:
            stats = get_stats(ids)
            pair = min(stats, key= lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids

In [ ]:
tok = BasicTokenizer()

In [ ]:
tok.train('hello!', 257)

In [ ]:
tok.merges

{(104, 101): 256}

In [ ]:
tok.decode([256,104])

'heh'

In [ ]:
tok.encode("hey")

[256, 121]

In [ ]:
tok.decode(tok.encode("hey jude"))

'hey jude'